In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#leemos
viajes = pd.read_csv("uber_drives.csv")
viajes.head(5)

,START_DATE,END_DATE,CATEGORY,START,STOP,MILES,PURPOSE
0,1/1/2016 21:11,1/1/2016 21:17,Business,Fort Pierce,Fort Pierce,5.1,Meal/Entertain
1,1/2/2016 1:25,1/2/2016 1:37,Business,Fort Pierce,Fort Pierce,5.0,NaN
2,1/2/2016 20:25,1/2/2016 20:38,Business,Fort Pierce,Fort Pierce,4.8,Errand/Supplies
3,1/5/2016 17:31,1/5/2016 17:45,Business,Fort Pierce,Fort Pierce,4.7,Meeting
4,1/6/2016 14:42,1/6/2016 15:49,Business,Fort Pierce,West Palm Beach,63.7,Customer Visit


In [ ]:
#info
viajes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1155 entries, 0 to 1154
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   START_DATE  1155 non-null   object 
 1   END_DATE    1155 non-null   object 
 2   CATEGORY    1155 non-null   object 
 3   START       1155 non-null   object 
 4   STOP        1155 non-null   object 
 5   MILES       1155 non-null   float64
 6   PURPOSE     653 non-null    object 
dtypes: float64(1), object(6)
memory usage: 63.3+ KB


In [ ]:
#reemplazar vacíos de PURPOSE
viajes["PURPOSE"].fillna("NONE", inplace = True)

In [ ]:
#quedarse solo con hora de START_DATE
st_date = viajes.loc[:,"START_DATE"]

In [ ]:
#importamos datetime
from datetime import datetime

In [ ]:
#loop
lista = []
for ele in st_date:
  #el formato debe ser el mismo al del string para que convierta
  ele_conver = datetime.strptime(ele,"%m/%d/%Y %H:%M")
  lista.append(ele_conver)

In [ ]:
#reemplazamos
viajes["START_DATE"] = lista

In [ ]:
#lo mismo pero para end_date
en_date = viajes.loc[:,"END_DATE"]
lista2 = []
#loop
for ele in en_date:
  ele_convert = datetime.strptime(ele,"%m/%d/%Y %H:%M")
  lista2.append(ele_convert)
viajes["END_DATE"] = lista2

In [ ]:
#Que purpose recorre más millas
viajes.groupby("PURPOSE")["MILES"].agg(["sum", "mean"]).sort_values(by = ["sum"], ascending = False)

,sum,mean
PURPOSE,,
NONE,4893.5,9.748008
Meeting,2851.3,15.247594
Customer Visit,2089.5,20.688119
Meal/Entertain,911.7,5.698125
Temporary Site,523.7,10.474000
Errand/Supplies,508.0,3.968750
Between Offices,197.0,10.944444
Commute,180.2,180.200000
Moving,18.2,4.550000


In [ ]:
#que viaje tuvo la mayor duración
#nueva columna
viajes["DURACION"] = viajes["END_DATE"] - viajes["START_DATE"]

In [ ]:
maximo = viajes.loc[:,"DURACION"].max()

In [ ]:
#filtro
viajes[viajes["DURACION"] == maximo]

,START_DATE,END_DATE,CATEGORY,START,STOP,MILES,PURPOSE,DURACION
776,2016-09-27 21:01:00,2016-09-28 02:37:00,Business,Unknown Location,Unknown Location,195.6,NONE,0 days 05:36:00


In [ ]:
#destinos frecuentes
viajes.loc[:,"STOP"].value_counts()
#viajes a CARY
v_cary=viajes[viajes["STOP"] == "Cary"]

In [ ]:
v_cary.loc[:,"MILES"].agg(["max","min"])

max    180.2
min      0.5
Name: MILES, dtype: float64

In [ ]:
#Puntos de inicio más frecuentes hacia el destino más frecuente (Cary)
v_cary.loc[:,"START"].value_counts()

Morrisville      75
Cary             53
Durham           32
Raleigh          21
Apex             13
Holly Springs     2
Chapel Hill       1
Florence          1
Boone             1
Wake Forest       1
Eagle Rock        1
Mebane            1
Fuquay-Varina     1
Name: START, dtype: int64

In [ ]:
v_cary.groupby("START")["MILES"].mean().sort_values(ascending = False)
#Morrisville es el punto de inicio más frecuente para llegar a Cary (Punto final más frecuente)
#Solo demora 5.276 miles en llegar

START
Boone            180.200000
Florence         159.300000
Mebane            45.200000
Wake Forest       31.900000
Chapel Hill       23.300000
Raleigh           17.414286
Fuquay-Varina     15.600000
Holly Springs     13.500000
Durham            10.450000
Apex               5.792308
Morrisville        5.276000
Cary               4.828302
Eagle Rock         3.600000
Name: MILES, dtype: float64

In [ ]:
#proposito comun de morrisville a cary
v_morri_Cary = v_cary[v_cary["START"] == "Morrisville"]
v_morri_Cary.loc[:,"PURPOSE"].value_counts()
#Propósito de viaje más frecuente de esta ruta es para comer/entretenimiento y visitas a consumidores(Su base está en Cary)?

NONE               24
Meal/Entertain     18
Customer Visit     18
Errand/Supplies     8
Meeting             3
Between Offices     2
Temporary Site      1
Moving              1
Name: PURPOSE, dtype: int64